实验结果汇总

|检索器|topK|hit_rate|mrr|precision|recall|ap|ndcg|备注|
|---|---|---|---|---|---|---|---|---|
|VectorStoreIndex|2|0.779751|0.708703|0.400829|0.779751|0.708703|0.727308||
|VectorStoreIndex|3|0.832445|0.725873|0.284784|0.832445|0.725873|0.753387||
|VectorStoreIndex|5|0.885731|0.73954|0.184162|0.885731|0.73954|0.776535||
|VectorStoreIndex|5|0.727649|0.628705|0.14998|0.727649|0.628705|0.653534|VectorStoreQueryMode.LINEAR_REGRESSION|
|VectorStoreIndex|5|0.787448|0.684192|0.16269|0.787448|0.684192|0.709495|VectorStoreQueryMode.MMR|
|SummaryIndex|5|0.971581|0.012225|0.001776|0.971581|0.012225|0.134562||
|BM25Retriever|2|0.092954|0.086738|0.048253|0.092954|0.086738|0.088366|官方|
|BM25Retriever|2|0.871522|0.806986|0.448194|0.871522|0.806986|0.823886|自定义|
|BM25Retriever+VectorStoreIndex|2,2,2|0.872114|0.809355|0.44701|0.872114|0.809355|0.825789|自定义+QueryFusionRetriever|
|BM25Retriever+VectorStoreIndex|3,5,2|0.870337|0.804914|0.435169|0.870337|0.804914|0.822046|自定义+QueryFusionRetriever|
|BM25Retriever+VectorStoreIndex|3,5,3|0.907046|0.81715|0.302349|0.907046|0.81715|0.8404|自定义+QueryFusionRetriever|
|BM25Retriever+VectorStoreIndex|3,5,3|0.910598|0.800276|0.303533|0.910598|0.800276|0.828861|FUSION_MODES.RECIPROCAL_RANK|
|BM25Retriever+VectorStoreIndex|3,5,3|0.910006|0.799092|0.303335|0.910006|0.799092|0.82779|num_queries=2|

## Setting

In [1]:
import os
import re

import nest_asyncio
nest_asyncio.apply()

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext,load_index_from_storage

from llama_index.core.evaluation import (
    generate_qa_embedding_pairs,
    EmbeddingQAFinetuneDataset
)

In [2]:
# 设置llm模型
llm_name="qwen2.5:latest"
embedding_name="quentinz/bge-large-zh-v1.5:latest"
base_url='http://localhost:11434'

Settings.llm = Ollama(
    model=llm_name, 
    request_timeout=360.0,
    base_url=base_url)

# 设置embedding model 
Settings.embed_model = OllamaEmbedding(
    model_name=embedding_name,
    base_url=base_url)

## Documents->nodes

In [3]:
import glob
from copy import deepcopy
from llama_index.core.schema import Document,TextNode

def split_markdown_by_headers(markdown_text):
    # 正则表达式匹配Markdown的一级和二级标题
    headers = re.compile(r'^(#+) (.*)$', re.MULTILINE)
    
    # 用于存储结果的列表
    headers_content = {}
    current_block = []
    current_title=''
    no_content_h1=''
    
    # 按行分割文档
    lines = markdown_text.split('\n')
    
    for line in lines:
        # 检查当前行是否是标题
        match = headers.match(line)
        if match:
            # 如果是标题，保存当前块（如果有的话）
            if current_block:
                if len(current_block)<=2:
                    no_content_h1=current_block[0]
                else:
                    headers_content[current_title]=no_content_h1+'\n'+''.join(current_block)
                    current_block=[]
            current_title=match.string
            
        # 如果不是标题，添加到当前块
        current_block.append(line + '\n')
    
    # 添加最后一个块
    headers_content[current_title]=''.join(current_block)
    
    return headers_content

def get_block_images(block):
    images_path = re.findall(r'!\[.*?\]\((.*?)\)', block)
    return images_path

def process_content(headers_content):
    nodes=[]
    
    for header in headers_content:
        # 获取block的图片
        block=headers_content[header]
        images_path=get_block_images(block)

        # 去掉block的图片文本
        pattern = r"!\[.*\)"
        block= re.sub(pattern, "\n", block)

        # 添加metadata：标题、内容等级、图片路径
        metadata={
            'title': block.split('\n')[0].replace('#',''),
            'content_level': header.count('#') ,
            'images_path': images_path
        }
        
 
        node=TextNode(
            text=block,
            metadata=deepcopy(metadata),
        )
        nodes.append(node)
    return nodes

def get_nodes_by_documents():
    documents=[]
    md_files=glob.glob('./preprocess/*.md')
    for md_file in md_files:
        with open(md_file,encoding='utf-8') as fr:
            md_content='\n'.join(fr.readlines())
        documents.append(md_content)
    
    nodes=[]
    for document in documents:
        headers_content=split_markdown_by_headers(document)
        document_nodes=process_content(headers_content)
        nodes.extend(document_nodes)
    
    return nodes


nodes=get_nodes_by_documents()
print(len(nodes),nodes[30])

563 Node ID: 2d8f4884-c654-4554-9652-5b4d7d3d9cc7
Text: ## 打开/关闭全景天窗   手动滑动打开（轻按按钮至第1个停止位置）。  自动滑动打开（按到底）。
手动滑动关闭（轻按按钮至第1个停止位置）。  自动滑动关闭（按到底）。  如果全景天窗和遮阳帘处于完全关闭状态，轻按控制按钮，先打开遮
阳帘，只有再次按下控制按钮后，才能打开全景天窗。  如果全景天窗和遮阳帘处于完全关闭状态，短时间内将控制按钮按到
底两次，遮阳帘和全景天窗同时打开。


## nodes->indexs

In [4]:
# 查看所有的Index
from llama_index.core import indices

indexs=list(filter(lambda att:att.find('Index')>0,dir(indices)))
print(indexs)

['DocumentSummaryIndex', 'EmptyIndex', 'GPTDocumentSummaryIndex', 'GPTEmptyIndex', 'GPTKeywordTableIndex', 'GPTListIndex', 'GPTPandasIndex', 'GPTRAKEKeywordTableIndex', 'GPTSQLStructStoreIndex', 'GPTSimpleKeywordTableIndex', 'GPTTreeIndex', 'GPTVectorStoreIndex', 'KeywordTableIndex', 'KnowledgeGraphIndex', 'ListIndex', 'MultiModalVectorStoreIndex', 'PandasIndex', 'PropertyGraphIndex', 'RAKEKeywordTableIndex', 'SQLStructStoreIndex', 'SimpleKeywordTableIndex', 'SummaryIndex', 'TreeIndex', 'VectorStoreIndex']


In [5]:
def build_index(IndexType,nodes,persist_dir):
    if os.path.exists(persist_dir):
        storage_context=StorageContext.from_defaults(persist_dir=persist_dir)
        index=load_index_from_storage(storage_context=storage_context)
    else:
        index=IndexType(nodes=nodes,show_progress=True)
        index.storage_context.persist(persist_dir=persist_dir)
    
    return index


## Evaluation Retriever

In [6]:
from llama_index.core.evaluation import RetrieverEvaluator
import random
import pandas as pd

def display_results(name,metrics, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)
    # print(metric_df)

    return metric_df

def generate_qa_by_nodes(nodes,save_json_path='pg_eval_dataset.json'):
    if os.path.exists(save_json_path):
        qa_dataset=EmbeddingQAFinetuneDataset.from_json(path=save_json_path)
    else:
        # 生成测试数据
        DEFAULT_QA_GENERATE_PROMPT_TMPL = """\
        Context information is below.

        ---------------------
        {context_str}
        ---------------------

        Given the context information and not prior knowledge.
        generate only questions based on the below query.

        You are a Teacher/ Professor. Your task is to setup \
        {num_questions_per_chunk} questions for an upcoming \
        quiz/examination. The questions should be diverse in nature \
        across the document. Restrict the questions to the context information provided.\
        最终提问用中文生成
        """

        qa_dataset=generate_qa_embedding_pairs(
            nodes=nodes,
            qa_generate_prompt_tmpl=DEFAULT_QA_GENERATE_PROMPT_TMPL,
            num_questions_per_chunk=3)
        qa_dataset.save_json(save_json_path)
    
    return qa_dataset

async def MyRetrieverEvaluator(retriever,qa_dataset):

    metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        metrics, retriever=retriever
    )

    # try it out on an entire dataset
    eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset,workers=8,show_progress=True)

    return (eval_results,metrics)

In [7]:
# 生成测试数据
qa_dataset=generate_qa_by_nodes(nodes,save_json_path='test_retriever_dataset.json')

## Test

### VectorStoreIndex

In [ ]:
from llama_index.core.indices import VectorStoreIndex

index_type=VectorStoreIndex
index_name=index_type.__name__
index=build_index(index_type,nodes,os.path.join('./Storage',index_name))

retriever = index.as_retriever(similarity_top_k=5 ,verbose=True)
(eval_results,metrics)= await MyRetrieverEvaluator(retriever,qa_dataset)

display_results(index_name,metrics,eval_results)

100%|██████████| 1689/1689 [00:42<00:00, 39.91it/s]


,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,VectorStoreIndex,0.885731,0.739836,0.184162,0.885731,0.739836,0.776753


### SummaryIndex

In [9]:
from llama_index.core.indices import SummaryIndex

index_type=SummaryIndex
index_name=index_type.__name__
index=build_index(index_type,nodes,os.path.join('./Storage',index_name))

retriever = index.as_retriever(similarity_top_k=5 ,verbose=True)
(eval_results,metrics)= await MyRetrieverEvaluator(retriever,qa_dataset)

display_results(index_name,metrics,eval_results)

100%|██████████| 1689/1689 [00:21<00:00, 78.65it/s]


,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,SummaryIndex,0.971581,0.012225,0.001776,0.971581,0.012225,0.134562


### BM25Retriever

In [10]:
import logging
import os
from llama_index.core.storage.docstore.types import BaseDocumentStore
from typing import Any, Callable, Dict, List, Optional, cast
from llama_index.core.callbacks.base import CallbackManager
from llama_index.core.constants import DEFAULT_SIMILARITY_TOP_K
from llama_index.core.schema import BaseNode, IndexNode, NodeWithScore, QueryBundle
from llama_index.core.vector_stores.utils import (
    node_to_metadata_dict,
    metadata_dict_to_node,
)

import itertools
import jieba
from llama_index.retrievers.bm25 import BM25Retriever


class ChineseBM25Retriever(BM25Retriever):
    """A BM25 retriever that uses the BM25 algorithm to retrieve nodes.

    Args:
        nodes (List[BaseNode], optional):
            The nodes to index. If not provided, an existing BM25 object must be passed.
        similarity_top_k (int, optional):
            The number of results to return. Defaults to DEFAULT_SIMILARITY_TOP_K.
        callback_manager (CallbackManager, optional):
            The callback manager to use. Defaults to None.
        objects (List[IndexNode], optional):
            The objects to retrieve. Defaults to None.
        object_map (dict, optional):
            A map of object IDs to nodes. Defaults to None.
        verbose (bool, optional):
            Whether to show progress. Defaults to False.
    """

    def _chinese_tokenizer(self, texts: List[str]) -> tuple[str]:
        # Use jieba to segment Chinese text
        rslts = tuple(itertools.chain.from_iterable(jieba.cut(text) for text in texts))
        return rslts

    def __init__(
            self,
            nodes: Optional[List[BaseNode]] = None,
            similarity_top_k: int = DEFAULT_SIMILARITY_TOP_K,
            callback_manager: Optional[CallbackManager] = None,
            objects: Optional[List[IndexNode]] = None,
            object_map: Optional[dict] = None,
            verbose: bool = False,
    ) -> None:

        super().__init__(
            nodes=nodes,
            similarity_top_k=similarity_top_k,
            callback_manager=callback_manager,
            objects=objects,
            object_map=object_map,
            verbose=verbose,
        )
        
        # change the stop words for Chinese
        with open(r'./stopwords-zh.txt', encoding='utf-8') as f: # here needs to add in the path of chinese stopwords
            con = f.readlines()
            stop_words = set()
            for i in con:
                i = i.rstrip('\n')
                stop_words.add(i)
        self.stop_words = stop_words

        corpus_tokens = [
            [word for word in jieba.cut_for_search(node.get_content()) if word not in stop_words and word.strip('\n')]
            for node in nodes
        ]
        corpus = [node_to_metadata_dict(node) for node in nodes]
        self.bm25.corpus = corpus
        self.bm25.index(corpus_tokens, show_progress=True)
    
    @classmethod
    def from_defaults(
        cls,
        index: Optional[VectorStoreIndex] = None,
        nodes: Optional[List[BaseNode]] = None,
        docstore: Optional[BaseDocumentStore] = None,
        similarity_top_k: int = DEFAULT_SIMILARITY_TOP_K,
        verbose: bool = False,
    ) -> "ChineseBM25Retriever":
          # ensure only one of index, nodes, or docstore is passed
        if sum(bool(val) for val in [index, nodes, docstore]) != 1:
            raise ValueError("Please pass exactly one of index, nodes, or docstore.")

        if index is not None:
            docstore = index.docstore

        if docstore is not None:
            nodes = cast(List[BaseNode], list(docstore.docs.values()))
        
        assert (
            nodes is not None
        ), "Please pass exactly one of index, nodes, or docstore."
      
        return cls(
            nodes=nodes,
            similarity_top_k=similarity_top_k,
            verbose=verbose,
        )
      
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query = query_bundle.query_str

        tokenized_query = [[word for word in jieba.cut_for_search(query) if word not in self.stop_words]]

        indexes, scores = self.bm25.retrieve(
            tokenized_query, k=self.similarity_top_k, show_progress=self._verbose
        )

        # batched, but only one query
        indexes = indexes[0]
        scores = scores[0]

        nodes: List[NodeWithScore] = []
        for idx, score in zip(indexes, scores):
            # idx can be an int or a dict of the node
            if isinstance(idx, dict):
                node = metadata_dict_to_node(idx)
            else:
                node_dict = self.corpus[int(idx)]
                node = metadata_dict_to_node(node_dict)
            nodes.append(NodeWithScore(node=node, score=float(score)))

        return nodes

resource module not available on Windows


c:\Users\wushaogui\miniconda3\envs\langchian\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from llama_index.core.node_parser import SentenceSplitter

index_type=VectorStoreIndex
index_name=index_type.__name__
index=build_index(index_type,nodes,os.path.join('./Storage',index_name))
retriever = ChineseBM25Retriever.from_defaults(
    index=index,
    similarity_top_k=2
)

(eval_results,metrics)= await MyRetrieverEvaluator(retriever,qa_dataset)

index_name='BM25Retriever'
display_results(index_name,metrics,eval_results)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\WUSHAO~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.438 seconds.
Prefix dict has been built successfully.
100%|██████████| 1689/1689 [00:00<00:00, 1773.98it/s]                   


,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,BM25Retriever,0.871522,0.806986,0.448194,0.871522,0.806986,0.823886


### QueryFusionRetriever=VectorStoreIndex+BM25Retriever

In [12]:
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.retrievers.fusion_retriever import FUSION_MODES

retriever = QueryFusionRetriever(
    [
        index.as_retriever(similarity_top_k=5),
        ChineseBM25Retriever.from_defaults(
            index=index,
            similarity_top_k=3
),
    ],
    num_queries=1,
    use_async=True,
    similarity_top_k=3,
    mode=FUSION_MODES.RECIPROCAL_RANK
)

(eval_results,metrics)= await MyRetrieverEvaluator(retriever,qa_dataset)

index_name='BM25Retriever'
display_results(index_name,metrics,eval_results)

100%|██████████| 1689/1689 [00:47<00:00, 35.67it/s]                     


,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,BM25Retriever,0.910598,0.800276,0.303533,0.910598,0.800276,0.828861
